In [2]:
from GA.geneticAlgorithm import GenerationalGA
from GA.parentSelector.parentSelector import RandomParentSelector, LinealOrder, TournamentSelection
from GA.parentSelector.parentSelector import WheelSelection, LinealOrderII
from utils.datamanager import DataManager
from utils.codificication_mlp import Layer, Cromosome, Fitness

import numpy as np
from time import time

Using TensorFlow backend.


In [8]:
from scipy import stats


class ParameterSensibilation(object):
    
    def __init__(self,chrom, cross_validation=False, iters=5, act=True, drop=True,
                 units=True, layers=True, delta=False):
        self.cross_val = cross_validation
        self.iters = iters
        self.act = act
        self.drop = drop
        self.units = units
        self.layers = layers
        self.chrom = chrom
        self.levels = 2
        self.delta = delta
        
        
    def start_tests(self):
        print("Testing Baseline")
        self.baseline_data = self.test_single(self.chrom)
        all_fits = []
        if self.act:
            all_fits.append(self.activation_test(self.chrom))
        if self.drop:
            all_fits.append(self.dropout_test(self.chrom, self.delta))
        if self.units:
            all_fits.append(self.units_test(self.chrom, self.delta))
        if self.layers:
            all_fits.append(self.layer_test(self.chrom))
        return all_fits
        
    def test_single(self, new_chrom):
        fits = np.ones(5)
        print(new_chrom, end='')
        if self.cross_val:
            fits = new_chrom.cross_val(exclude_first=False)
        else:
            fits = [new_chrom.fitness() for i in range(self.iters)]
        print("Fitness: %0.4f (%0.4f)" % (np.mean(fits), np.std(fits)))
        return fits
    
    
    def test_samples(self, new_chroms):
        for new_chrom in new_chroms:
            fits = self.test_single(new_chrom)
            t_value, p_value = stats.ttest_ind(self.baseline_data, fits)
            print("t = %0.4f, p = %0.4f\n" % (t_value, p_value))
        return fits
        
    
    def activation_test(self, chrom, crossval=True):
        print("\nActivation test:\n")
        new_chroms = []
        all_relu = chrom.self_copy()
        for n in range(len(chrom.layers)):
            new_chrom = chrom.self_copy()
            new_chrom.layers[n].activation = 'relu'
            new_chroms.append(new_chrom)
            all_relu.layers[n].activation = 'relu'
        new_chroms.append(all_relu)
        return self.test_samples(new_chroms)


    def dropout_test(self, chrom, crossval=False, delta=False):

        def get_drop(d, i):
            if i < 0:
                return d - d * (2 ** i)
            else:
                return d + (1 - d) * (0.5 ** (i + 1))

        print("\nDropout test\n")
        new_chroms = [chrom.self_copy() for i in range(len(chrom.layers))]
        if delta:
            new_chroms = [chrom.self_copy() for i in range(2 * self.levels)]
        dropout_zero = chrom.self_copy()
        dropout_02 = chrom.self_copy()
        for n in range(len(chrom.layers)):
            dropout_zero.layers[n].dropout = 0
            dropout_02.layers[n].dropout = 0.2
            if delta:
                for i in range(len(new_chroms)):
                    d = new_chroms[i].layers[n].dropout
                    new_chroms[i].layers[n].dropout = get_drop(d, i - self.levels)
            else:
                new_chroms[n].layers[n].dropout = 0.2
        new_chroms.append(dropout_zero)
        new_chroms.append(dropout_02)
        return self.test_samples(new_chroms)

    
    def layer_test(self, chrom):
        print("\nLayer test\n")
        new_chroms = [chrom.self_copy()]
        for i in range(len(chrom.layers)):
            original_chrom = new_chroms[-1].self_copy()
            new_layers = original_chrom.layers
            new_layers.pop()
            new_chroms.append(Cromosome(new_layers))
        return self.test_samples(new_chroms)


    def units_test(self, chrom, delta=True):
        def get_units(d, i):
            if i < 0:
                return int(d * (1 + i * 0.2))
            else:
                return int(d * (1 + (i + 1) * 0.2))

        print("\nUnits test\n")
        new_chroms = [chrom.self_copy() for i in range(len(chrom.layers))]
        if delta:
            new_chroms = [chrom.self_copy() for i in range(2 * self.levels)]
        
        units_512 = chrom.self_copy()
        for n in range(len(chrom.layers)):
            units_512.layers[n].units = 512
            if delta:
                for i in range(2 * self.levels):
                    d = new_chroms[i].layers[n].units
                    new_chroms[i].layers[n].units = get_units(d, i - self.levels)
            else:
                new_chroms[n].layers[n].units = 512
        new_chroms.append(units_512)
        return self.test_samples(new_chroms)

In [13]:
l = Layer(512, 'relu', 0.2)
l2 = Layer(512, 'relu', 0.2)
benchm = Cromosome([l, l2])

ps = {'random':RandomParentSelector(), 'lineal':LinealOrder(), 'wheel':WheelSelection(), 
      'tournament':TournamentSelection(5)}

# dataset params:
dataset = 'mnist'
classes = []

# genetic algorithm params:
parents_selector_key = 'wheel'
num_parents = 0.3
generations = 40
population = 15
train_time = 10

# Fitness params
epochs = 75
batch_size = 128
maximize_fit = True
verbose = 0
redu_plat = False
early_stop = True

# Load data
dm = DataManager(dataset, clases=classes)
data = dm.load_data()
fitness = Fitness.get_instance()
fitness.set_params(data, batch_size=batch_size, verbose=verbose, reduce_plateau=redu_plat, 
                   epochs=epochs, early_stop=early_stop)
p = ps[parents_selector_key]

(48000, 28, 28, 1) train samples
(12000, 28, 28, 1) validation samples
(10000, 28, 28, 1) test samples


In [ ]:
ti_all = time()
generational = GenerationalGA(num_parents=num_parents, chromosome=benchm, parent_selector=p,
                              generations=generations, num_population=population, crossover_prob=0.5,
                              mutation_prob=0.7, maximize_fitness=maximize_fit, training_hours=train_time)
winner, best_fit, ranking = generational.evolve()
print("Total elapsed time: %0.3f" % (time() - ti_all))
print("Total training time: %0.3f" % fitness.time)

In [6]:
winner = Cromosome([Layer(830, 'prelu',0.654), Layer(782, 'tanh', 0.166), Layer(28, 'elu', 0.062)])
benchm = Cromosome([Layer(512, 'relu', 0.2), Layer(512, 'relu', 0.2)])

In [14]:
ps = ParameterSensibilation(Cromosome([Layer(830, 'prelu', 0.654)]), 
                            drop=True, layers=False, act=True, iters=10, delta=True)
fits = ps.start_tests()

Testing Baseline
0 - U:830|A:prelu|D:0.654 
Fitness: 0.9840 (0.0006)

Activation test:

0 - U:830|A:relu|D:0.654 
Fitness: 0.9841 (0.0007)
t = -0.1443, p = 0.8888

0 - U:830|A:relu|D:0.654 
Fitness: 0.9835 (0.0005)
t = 1.2805, p = 0.2362


Dropout test

0 - U:830|A:prelu|D:0.200 
Fitness: 0.9831 (0.0003)
t = 2.7809, p = 0.0239

0 - U:830|A:prelu|D:0.000 
Fitness: 0.9842 (0.0005)
t = -0.4491, p = 0.6653

0 - U:830|A:prelu|D:0.200 
Fitness: 0.9827 (0.0008)
t = 2.6370, p = 0.0299


Units test

0 - U:498|A:prelu|D:0.654 
Fitness: 0.9836 (0.0003)
t = 1.1070, p = 0.3005

0 - U:664|A:prelu|D:0.654 
Fitness: 0.9835 (0.0006)
t = 1.1767, p = 0.2731

0 - U:996|A:prelu|D:0.654 
Fitness: 0.9845 (0.0003)
t = -1.2430, p = 0.2490

0 - U:1162|A:prelu|D:0.654 
Fitness: 0.9843 (0.0004)
t = -0.8619, p = 0.4139

0 - U:512|A:prelu|D:0.654 
Fitness: 0.9836 (0.0004)
t = 1.0839, p = 0.3100



In [11]:
ps = ParameterSensibilation(winner, act=False, drop=False, layers=True, units=False, iters=5)
ps.start_tests()

Testing Baseline
0 - U:830|A:prelu|D:0.654 
1 - U:782|A:tanh|D:0.166 
2 - U:28|A:elu|D:0.062 
Fitness: 0.9838 (0.0006)

Units test

0 - U:512|A:prelu|D:0.654 
1 - U:782|A:tanh|D:0.166 
2 - U:28|A:elu|D:0.062 
Fitness: 0.9826 (0.0007)
t = 2.4607, p = 0.0393

0 - U:830|A:prelu|D:0.654 
1 - U:512|A:tanh|D:0.166 
2 - U:28|A:elu|D:0.062 
Fitness: 0.9842 (0.0007)
t = -0.9141, p = 0.3874

0 - U:830|A:prelu|D:0.654 
1 - U:782|A:tanh|D:0.166 
2 - U:512|A:elu|D:0.062 
Fitness: 0.9845 (0.0004)
t = -1.9158, p = 0.0917

0 - U:512|A:prelu|D:0.654 
1 - U:512|A:tanh|D:0.166 
2 - U:512|A:elu|D:0.062 
Fitness: 0.9828 (0.0007)
t = 1.9620, p = 0.0854



In [ ]:
fitness.calc_mean(winner)

In [ ]:
fitness.calc_mean(benchm)

In [3]:
1500/3600

0.4166666666666667